In [1]:
import copy
import math

import numpy as np

import torch 
from torch import nn

### **PLOT**

In [27]:
from matplotlib import pyplot as plt

In [28]:
class plot:
  def __init__(this, X, Y=None, xlabel=None, ylabel=None, legend=[],
               xlim=None, ylim=None, xscale='linear', yscale='linear', axes=None):

    def isHas1axis(X):  # True if `X` (tensor or list) has 1 axis
        return (hasattr(X, "ndim") and X.ndim == 1 or isinstance(X, list) and not hasattr(X[0], "__len__"))

    if isHas1axis(X): X = [X]
    if Y is None: X, Y = [[]] * len(X), X
    if isHas1axis(Y):Y = [Y]
    if len(X) != len(Y): X = X * len(Y)

    plt.rcParams['figure.figsize'] = (7,5)
    if axes is None: 
      axes = plt.gca()#get get current axes
    axes.cla() #Clear the current active axes.
    for x, y in zip(X, Y):
      if len(x):axes.plot(x,y)
      else: axes.plot(y)
        #axes.plot(x,y) if len(x) else axes.plot(y)
    this.set_axes(axes, xlabel, ylabel, xlim, ylim, xscale, yscale, legend)

    
  def set_axes(this, axes, xlabel, ylabel, xlim, ylim, xscale, yscale, legend):
    axes.set_xlabel(xlabel)
    axes.set_ylabel(ylabel)
    axes.set_xscale(xscale)
    axes.set_yscale(yscale)
    axes.set_xlim(xlim)
    axes.set_ylim(ylim)
    
    if legend:
        axes.legend(legend)
    #axes.grid()

  def set_figsize(this, figsize=(10, 2.5)):
     plt.figure.figsize = figsize

In [29]:
def plotImages(imgs, r, c, titles=None, scale=1.5):
  figsize = (c * scale, r * scale)

  _, axes = plt.subplots(r, c, figsize=figsize)
  
  axes = axes.flatten()
  for i, (ax, img) in enumerate(zip(axes, imgs)):
    if torch.is_tensor(img):
      ax.imshow(img.numpy())
    else:
      ax.imshow(img)
      ax.axes.get_xaxis().set_visible(False)
      ax.axes.get_yaxis().set_visible(False)
    if titles:
      ax.set_title(titles[i])
  return axes

### **Data**

In [30]:
import torchvision
from torch.utils import data
from torchvision import transforms

In [31]:
def getFashionMNIST(batchSize, resize = None):
  ts = []
  if resize:
    ts.append(transforms.Resize(resize))
  ts.append(transforms.ToTensor())#transfrom the values of a tensor(C * H * W) from [0, 255] to [0, 1]
  tsCombo = transforms.Compose(ts) 
  mnist_train = torchvision.datasets.FashionMNIST(root="../data", train=True, transform = tsCombo, download=True)
  mnist_test  = torchvision.datasets.FashionMNIST(root="../data", train=False, transform = tsCombo, download=True)
  _X = data.DataLoader(mnist_train, batchSize, shuffle=True, num_workers= 2)
  _y = data.DataLoader(mnist_test, batchSize, shuffle=False, num_workers= 2)
  return _X, _y

In [32]:
def get_labels(labels):
  if not hasattr(labels, "ndim") and not hasattr(labels, "__len__"): labels =  numpy.array([labels])
  text_labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat', 'sandal', 'shirt','sneaker', 'bag', 'ankle boot']
  return [text_labels[int(i)] for i in labels]

In [33]:
# def randomSimple(size):
#   return next(iter(getFashionMNIST(size)[0]))

In [34]:
# X_, y_ = randomSimple(18)
# plotImages(X_.reshape(18, 28, 28), 3, 3, titles=get_labels(y_),scale=3)

In [35]:
def batch_iterator(X, y=None, batch_size=64):
  n_samples = X.shape[0]
  for i in np.arange(0, n_samples, batch_size):
    begin, end = i, min(i+batch_size, n_samples)
    if y is not None:
      yield X[begin:end], y[begin:end]
    else:
      yield X[begin:end]

### **PyTorch Version**

In [21]:
class NN(nn.Module):
  
  def __init__(this, inSize, outSize):
    super().__init__()
    this.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(inSize,inSize*2),
        nn.ReLU(),

        nn.Linear(inSize*2,inSize*7),
        nn.ReLU(),
        
        nn.Linear(inSize*7,inSize*7),
        nn.ReLU(),

        nn.Linear(inSize*7,inSize*7),
        nn.ReLU(),

        nn.Linear(inSize*7,inSize*7),
        nn.ReLU(),

        nn.Linear(inSize*7,inSize*7),
        nn.ReLU(),

        nn.Linear(inSize*7,outSize),
        nn.Sigmoid(),
    )

  def forward(this, X):
    return  this.classifier(X)


### **TEST NN**

In [22]:
from NN.NN import NN
from NN.Optimisation.Adam import Adam
from NN.Layer.Dense import Dense
from NN.Layer.Activation import Activation 
from NN.Loss.SquareLoss import SquareLoss

from sklearn import datasets
from sklearn.model_selection import train_test_split

In [23]:
def accuracy_score(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred, axis=0) / len(y_true)
    return accuracy

In [24]:
def oneHot(x, nbrCol=None):
  if not nbrCol: nbrCol = np.max(x) + 1
  one_hot = np.zeros((x.shape[0], nbrCol))
  one_hot[np.arange(x.shape[0]), x] = 1
  return one_hot

In [ ]:
# data = getFashionMNIST(batchSize=500,resize=8)

In [38]:
X_, y_ = next(iter(data[0]))

In [39]:
X = []
for i in X_:
  X.append(torch.reshape(i,(64,)).cpu().detach().numpy())
X = np.array(X)

y = oneHot(y_.cpu().detach().numpy())

In [40]:
optimizer = Adam()
n_hidden = 100
n_samples, n_features = X.shape
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=72)

In [43]:
mlp = NN_(optimizer= optimizer,loss= SquareLoss,validationData=(X_test, y_test))
mlp.add(Dense(n_hidden, inputShape=(n_features, )))
mlp.add(Activation('relu'))
mlp.add(Dense(10))
mlp.add(Activation('sigmoid'))
train_err, val_err = mlp.fit(X_train, y_train, n_epochs=200, batch_size=256)


done!!


In [44]:
yp = np.argmax(mlp.predict(X_test), axis=1)
yt = np.argmax(y_test, axis=1)
accuracy_score(y_true=yt,y_pred=yp)

0.6266666666666667

### **TEST pytorch**

In [45]:
from torch.autograd import Variable

In [47]:
from torchsummary import summary
model = NN(inSize=64 ,outSize=10)
summary(model,(1,8,8))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                   [-1, 64]               0
            Linear-2                  [-1, 128]           8,320
              ReLU-3                  [-1, 128]               0
            Linear-4                  [-1, 448]          57,792
              ReLU-5                  [-1, 448]               0
            Linear-6                  [-1, 448]         201,152
              ReLU-7                  [-1, 448]               0
            Linear-8                  [-1, 448]         201,152
              ReLU-9                  [-1, 448]               0
           Linear-10                  [-1, 448]         201,152
             ReLU-11                  [-1, 448]               0
           Linear-12                  [-1, 448]         201,152
             ReLU-13                  [-1, 448]               0
           Linear-14                   

In [48]:
LossFun   = nn.CrossEntropyLoss()
optimizer = torch.optim.Adagrad(model.parameters(), lr = 0.001)

In [ ]:
# X,y = getFashionMNIST(batchSize=100)

In [52]:
X__,y__ = data

In [53]:
total_step = len(X__)
for epoch in range(5):
    for i, (images,labels) in enumerate(X__):
        images = Variable(images.float())
        labels = Variable(labels)
        
        # Forward pass
        outputs = model(images)
        loss = LossFun(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, 5, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/120], Loss: 1.8838
Epoch [2/5], Step [100/120], Loss: 1.8384
Epoch [3/5], Step [100/120], Loss: 1.8496
Epoch [4/5], Step [100/120], Loss: 1.8106
Epoch [5/5], Step [100/120], Loss: 1.8185


In [54]:
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in y__:
        images = Variable(images.float())
        labels = Variable(labels)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

In [55]:
print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 21.62 %
